## 1、字符串解析器 StrOutputParser

In [2]:
# 1、获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY1')
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 2.调用大模型
response = llm.invoke("什么是爱情？")
print(type(response))

# 3.使用字符串解析器
parser = StrOutputParser()
parsed_response = parser.invoke(response)
print(type(parsed_response))
print(parsed_response)

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>
爱情是一种复杂而深刻的情感，通常涉及对另一个人的强烈依恋、关心和吸引。它可以表现为浪漫的情感，也可以是亲情、友情等多种形式的深厚情感。爱情不仅仅是生理上的吸引，还包括心理和情感上的连接。

在不同的文化和哲学中，爱情的定义和表现形式各不相同。它可以带来快乐和满足感，也可能伴随着痛苦和挑战。爱情常常涉及信任、理解、包容和支持，是人际关系中重要的一部分。

总的来说，爱情是一种让人感到充实和有意义的情感体验，能够激励人们追求更好的自己和更美好的生活。


## 2、JSON解析器 JsonOutputParser
实现方式
- 方式1：用户自己通过提示词指明返回Json格式
- 方式2：借助JsonOutputParser的 get_format_instructions() ，生成格式说明，指导模型输出JSON 结构

In [6]:
# 方式1：
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_prompt_template = ChatPromptTemplate.from_messages([
	("system", "你是一个靠谱的{role}"),
	("human", "{question}")
])

result = chat_model.invoke(
	chat_prompt_template.invoke(
		{"role": "人工智能专家",
		 "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式"
		 }
	)
)
print(result.content)
print(type(result))

# 获取一个JsonOutputParser的实例
json_parser = JsonOutputParser()
parsed_result = json_parser.invoke(result)
print(parsed_result)
print(type(parsed_result))

```json
{
  "q": "人工智能用英文怎么说？",
  "a": "Artificial Intelligence"
}
```
<class 'langchain_core.messages.ai.AIMessage'>
{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}
<class 'dict'>


In [7]:
# 举例2
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Return a JSON object.


In [8]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
joke_query = "告诉我一个笑话。"
# 定义Json解析器
parser = JsonOutputParser()
# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
	template="回答用户查询的问题\n满足格式为{format_instructions}\n问题是:{question}",
	partial_variables={
		"format_instructions": parser.get_format_instructions(),
	}
)
prompt = prompt_template.invoke(input={"question": joke_query})
# 调用大模型
response = chat_model.invoke(prompt)
#
json_result = parser.invoke(response)
print(json_result)
print(type(json_result))

{'joke': '为什么自行车不能站起来？因为它有两个轮子！'}
<class 'dict'>


知识拓展：

In [9]:
# 针对举例1
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_prompt_template = ChatPromptTemplate.from_messages([
	("system", "你是一个靠谱的{role}"),
	("human", "{question}")
])

# 获取一个JsonOutputParser的实例
json_parser = JsonOutputParser()

chain = chat_prompt_template | chat_model | json_parser
json_result = chain.invoke(
	{"role": "人工智能专家",
	 "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式"
	 }
)
print(json_result)
print(type(json_result))

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}
<class 'dict'>


In [11]:
# 针对举例2
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
joke_query = "告诉我一个脑经急转弯。"

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
	template="回答用户查询的问题\n满足格式为{format_instructions}\n问题是:{question}",
	partial_variables={
		"format_instructions": parser.get_format_instructions(),
	}
)
# 定义Json解析器
parser = JsonOutputParser()

chain = prompt_template | chat_model | parser
json_result = chain.invoke(input={"question": joke_query})
print(json_result)
print(type(json_result))

{'riddle': '什么东西越洗越脏？', 'answer': '水'}
<class 'dict'>
